## Running the Roman pipeline

##### Before running this notebook, make sure you have gone through the setup instructions on the readme for this repository to set up CRDS, create a conda environment, and install the romancal pipline in this environment.
This notebook shows the basic commands to run the Roman pipeline in Python.

### Code development

There are several packages which are concurrently developed to support the Roman pipeline calibrations.
Some are shared with the JWST pipeline.

- rad: Roman schemas

https://github.com/spacetelescope/rad

- roman_datamodels: Datamodels for Roman data products

https://github.com/spacetelescope/roman_datamodels

- stpipe: Pipeline runner (shared with jwst)

https://github.com/spacetelescope/stpipe

- stcal: Core algorithms used in the pipeline (shared with jwst)

https://github.com/spacetelescope/stcal

- romancal: The actual pipeline code
https://github.com/spacetelescope/romancal

Installing romancal installs all its dependencies automatically.

### Opening a data file as a data model

In [ ]:
from astropy.utils.data import download_file

box_path = 'https://data.science.stsci.edu/redirect/Roman/Roman_Calibration_And_Datamodels/'
level1_file = box_path + "r0000101001001001001_01101_0001_WFI01_uncal.asdf"

level2_file = box_path + "r0000101001001001001_01101_0001_WFI01_cal.asdf"

In [ ]:
from roman_datamodels import datamodels as rdm

l1_file = download_file(level1_file)

data_model = rdm.open(l1_file)

In [ ]:
data_model

This is a special data model. Science data formatting creates a Level1 (`ScienceRawModel`) file, which is the input to the pipeline. 

The first step in the pipeline, `dqinit` needs to convert this to a `RampModel`. 

### Running a step using reference files in CRDS

#### On the command line

```
strun romancal.step.FlatFieldStep <input_file>
```

This runs the flat field step and saves the output in the current directory. The file suffix is `flatfieldstep`.

The original `rate` suffix is preserved. *This is something that needs to be fixed!*

#### In the Python interpreter

In [ ]:
from romancal.flatfield import FlatFieldStep

l2_file = download_file(level2_file)
result = FlatFieldStep.call(l2_file)

In this case the output `result` is an ImageModel and can be saved to file.

In [ ]:
print(result)

In [ ]:
result.save('my_flat_fielded_image.asdf')

### Running a step with local reference files

#### On the command line

```
strun romancal.step.FlatFieldStep <input_file> --override_flat=myflat.asdf
```


#### In the Python interpreter

In [ ]:
from romancal.flatfield import FlatFieldStep

result = FlatFieldStep.call(level2_file, override_flat="flat.asdf")

In general, to run the pipeline from the interpreter and pass custom reference files, one needs to use an argument name 

`override_<reference_file_type>=<file_name>`

On the command line the syntax is 

`--override_<reference_file_type>=<file_name>`

To find out what reference types a step uses and their names:

In [ ]:
from romancal.step import *

In [ ]:
print("flat_field: ", FlatFieldStep.reference_file_types)
print("dq_init: ", DQInitStep.reference_file_types)

In terms of code development, a "pipeline" is a special "step", which runs other steps. This means the Exposure Level Pipeline can be run in the same way as an individual step.

```
strun romancal.pipeline.ExposurePipeline r0000101001001001001_01101_0001_WFI01_uncal.asdf
```

or using the pipeline alias, `roman_elp`,

```
strun roman_elp r0000101001001001001_01101_0001_WFI01_uncal.asdf
```

The syntax to run the pipeline in the Python interpreter is also the same as for an individual step.

To list all available steps (note the list contains steps from all packages registered with stpipe):

In [ ]:
stpipe list